In [29]:
import pandas as pd
import nltk

In [30]:
x_train = pd.read_csv('training_twitter_x_y_train.csv')
x_test = pd.read_csv('test_twitter_x_test.csv')

In [31]:
Y_train = x_train['airline_sentiment']
X_train = x_train['text']
X_test = x_test['text']

In [32]:
X_train

In [34]:
from nltk.tokenize import sent_tokenize , word_tokenize

In [ ]:
training_data_x = []
for i in X_train:
    training_data_x.append(word_tokenize(i))

In [36]:
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
lemmatizer = WordNetLemmatizer()

In [45]:
from nltk.corpus import wordnet
def get_simple_pos(tag):
    
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN 
    

In [46]:
from nltk.corpus import stopwords
import string
stops = set(stopwords.words('english'))
punctuations = list(string.punctuation)
stops.update(punctuations)

In [47]:
def clean_review(words):
    output_words = []
    for w in words:
        if w.lower() not in stops:
            pos = pos_tag([w])
            clean_word = lemmatizer.lemmatize(w, pos = get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    return output_words

In [48]:
x_fin_train = [clean_review(w) for w in training_data_x]

In [52]:
x_fin_train[455]

['united',
 'receive',
 'one',
 'previously',
 '11th',
 'checked',
 'odd',
 'say',
 'complaint',
 'associate',
 'e-mail']

In [54]:
X_TRAIN = [" ".join(i) for i in x_fin_train]
Y_TRAIN = Y_train

In [55]:
X_TRAIN[0] , Y_TRAIN[0]

('southwestair schedule morning 2 day fact yes .. sure even flight one cancelled flightled',
 'negative')

In [56]:
from sklearn.feature_extraction.text import CountVectorizer

In [57]:
count_vec = CountVectorizer(max_features = 20000 , ngram_range=(1,3))
x_train_features = count_vec.fit_transform(X_TRAIN)
x_train_features.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [58]:
len(count_vec.get_feature_names())

20000

In [59]:
x_test_features = count_vec.transform(X_test)

In [60]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()

In [61]:
rfc.fit(x_train_features , Y_TRAIN)

RandomForestClassifier()

In [62]:
rfc.score(x_train_features , Y_TRAIN)

0.9946265938069216

In [63]:
y_test_pred = rfc.predict(x_test_features)

In [65]:
df = pd.DataFrame (y_test_pred)

In [66]:
df.to_csv('Twitter_analysis.csv', header=False, index=False)